### Reading from input file

In [1]:
print("There are several test network to try from. Please select one!")
print("1. roadNet-PA.txt")
print("2. roadNet-CA.txt")
print("3. mini_input.txt (randomly generated network with 100 nodes)")

choice = int(input("Insert the number (1-3): "))

if choice == 1:
    file = "roadNet-PA.txt"
elif choice == 2:
    file = "roadNet-CA.txt"
elif choice == 3:
    file = "mini_input.txt"
else:
    file = "mini_input.txt"

There are several test network to try from. Please select one!
1. roadNet-PA.txt
2. roadNet-CA.txt
3. mini_input.txt (randomly generated network with 100 nodes)
Insert the number (1-3): 1


In [2]:
# Parse the nodes input file into list of edge tuples
import re

with open(file) as f:
    next(f)
    next(f)
    line = f.readline()
    if choice == 1 or choice == 2:
        nodes_count = int(re.search(r'Nodes: \d+', line).group()[7:]) + 5000 # the number of nodes in the .txt is wrong
    else:
        nodes_count = int(re.search(r'Nodes: \d+', line).group()[7:])
    next(f)
    graph = []
    for line in f:
        graph.append(tuple(map(int, tuple(line.replace('\n', '').split('\t'))))) # formatting the list of tuples

In [3]:
hospital_count = -1
while hospital_count < 1:
    print("The number of hospital must be an integer between 1 and the number of nodes.")
    hospital_count = int(input("How many hospitals are there? "))

The number of hospital must be an integer between 1 and the number of nodes.
How many hospitals are there? 10000


### Generate random hospitals based on user input
#### <font color = red> If hospital.txt already exists, only run the third block!

In [4]:
# Generate a list of different random numbers
import random
random_list=[]
while(len(random_list) < hospital_count):
    r = random.randint(0, nodes_count-1)
    if r not in random_list: random_list.append(r)
        
# random_list

In [5]:
# Write the hospitals into a .txt file
f = open("hospital.txt", "w")
f.write("# " + str(hospital_count) + "\n")
for hospital in random_list:
    f.write(str(hospital) + "\n")
f.close()

In [6]:
# Parse the hospital data from an existing input file
hospital_list = []

hospital = open("hospital.txt", "r")

hospital.readline()
for unit in hospital:
    hospital_list.append(int(unit))
    
hospital_list

[766073,
 880122,
 784024,
 621372,
 792335,
 516215,
 599369,
 314706,
 803302,
 23410,
 582629,
 918136,
 126343,
 505890,
 58123,
 443839,
 291077,
 695737,
 849985,
 916819,
 612776,
 840142,
 473075,
 603444,
 392704,
 263444,
 431494,
 452155,
 233518,
 1070401,
 420699,
 1044421,
 56380,
 644730,
 137824,
 786156,
 356773,
 125086,
 159166,
 271258,
 1000093,
 640373,
 882441,
 212653,
 402149,
 33735,
 1043134,
 88603,
 231053,
 165280,
 615768,
 169836,
 886729,
 971243,
 498730,
 982729,
 992196,
 512591,
 487286,
 698479,
 954732,
 398619,
 314160,
 629002,
 744438,
 860999,
 620633,
 597103,
 755736,
 71129,
 294195,
 970102,
 611924,
 9839,
 366805,
 110439,
 75433,
 215446,
 177273,
 112678,
 806618,
 732781,
 272027,
 649684,
 1041472,
 794853,
 446998,
 38115,
 517079,
 449082,
 280143,
 658231,
 460589,
 1006125,
 981266,
 958389,
 351916,
 398458,
 749775,
 881324,
 632179,
 912786,
 958133,
 119313,
 358413,
 1092393,
 688388,
 943634,
 736599,
 550873,
 1021785,
 75

### Pre-processing stage
##### Listing all the nodes a node is connected to. Note that each edge is undirected. Hence, if A and B are connected, A -> B and B -> A apply.

In [7]:
%%time
# Create bi-directional edges
edges_dict = {edge_list: [] for edge_list in range(nodes_count)} 

for edge in graph:
    edges_dict[edge[0]].append(edge[1])
    edges_dict[edge[1]].append(edge[0])
    
# edges_dict

Wall time: 2.06 s


### Part(a). Time complexity depends on the number of hospitals

In [ ]:
%%time
def bfs(visited, graph, node):
    queue = [] # Initialize a queue
    
    visited.append(node)
    queue.append(node)
    
    if node in hospital_list:
        return node

    while queue:
        s = queue.pop(0) 
        # print (s, end = " ") 

        for neighbour in graph[s]:
            if neighbour not in visited:
                # print(neighbour)
                visit_dict[neighbour].append(s)
                if neighbour in hospital_list:
                    # print("Hospital is found!")
                    return neighbour
                visited.append(neighbour)
                queue.append(neighbour)
                
f = open("output(a).txt", "w")
for unit in range(nodes_count):
    # Driver Code
    visited = [] # List to keep track of visited nodes.
    count = 0
    visit_dict = {edge_list: [] for edge_list in range(nodes_count)} # store the parent node of every child node
    nearest = bfs(visited, edges_dict, unit)
    
    if nearest:
        point = nearest
        path = []

        path.append(point)

        while visit_dict[point] != []:
            point = visit_dict[point][0]
            path.append(point)
            count += 1

        if count != 0:
            f.write(str(path.pop()))
            while path:
                f.write(" -> ")
                f.write(str(path.pop()))
            f.write("\nThe distance to the nearest hospital is " + str(count) + " node(s).\n")
        else:
            f.write(str(path.pop()))
            f.write("\nIt is the hospital.\n")
    
    else:
        f.write(str(unit) + "\n")
        f.write("It is not connected to any hospitals.\n")
        
    f.write("\n")
f.close()

### Part(b). Time complexity does not depend on the number of hospitals
##### Additional pre-processing stage where each node is marked as TRUE/FALSE according to whether it is a hospital

In [ ]:
# Mark each node as a hospital (TRUE) or not hospital (FALSE)
hospital_flag = [False] * nodes_count
for flag in hospital_list:
    hospital_flag[flag] = True
# hospital_flag

In [ ]:
%%time
def bfs(visited, graph, node):
    queue = [] # Initialize a queue
    
    visited.append(node)
    queue.append(node)
    
    if hospital_flag[node]:
        return node

    while queue:
        s = queue.pop(0) 
        # print (s, end = " ") 

        for neighbour in graph[s]:
            if neighbour not in visited:
                # print(neighbour)
                visit_dict[neighbour].append(s)
                if hospital_flag[neighbour]:
                    # print("Hospital is found!")
                    return neighbour
                visited.append(neighbour)
                queue.append(neighbour)
                
f = open("output(b).txt", "w")
for unit in range(nodes_count):
    # Driver Code
    visited = [] # List to keep track of visited nodes.
    count = 0
    visit_dict = {edge_list: [] for edge_list in range(nodes_count)} # store the parent node of every child node
    nearest = bfs(visited, edges_dict, unit)
    
    if nearest:
        point = nearest
        path = []

        path.append(point)

        while visit_dict[point] != []:
            point = visit_dict[point][0]
            path.append(point)
            count += 1

        if count != 0:
            f.write(str(path.pop()))
            while path:
                f.write(" -> ")
                f.write(str(path.pop()))
            f.write("\nThe distance to the nearest hospital is " + str(count) + " node(s).\n")
        else:
            f.write(str(path.pop()))
            f.write("\nIt is the hospital.\n")
    
    else:
        f.write(str(unit))
        f.write("\nIt is not connected to any hospitals.\n")
    
    f.write("\n")
f.close()

### Part(c). Instead of one hospital, the algorithm will output the top-2 nearest hospital from every node

In [ ]:
%%time
def bfs(visited, graph, node, nearest_hospital):
    queue = [] # Initialize a queue
    
    visited.append(node)
    queue.append(node)
    
    if hospital_flag[node]:
        return 1

    while queue:
        s = queue.pop(0) 
        # print (s, end = " ") 

        for neighbour in graph[s]:
            if neighbour not in visited:
                # print(neighbour)
                visit_dict[neighbour].append(s)
                if hospital_flag[neighbour]:
                    nearest_hospital.append(neighbour)
                    if len(nearest_hospital) == 2:
                        return 0
                visited.append(neighbour)
                queue.append(neighbour)
    return 0
                
f = open("output(c).txt", "w")

for unit in range(nodes_count):
    # Driver Code
    visited = [] # List to keep track of visited nodes.
    visit_dict = {edge_list: [] for edge_list in range(nodes_count)} # store the parent node of every child node
    nearest_hospital = []
    flag = bfs(visited, edges_dict, unit, nearest_hospital)
    
    f.write(str(unit) + "\n")
    
    if flag == 1:
        f.write("It is the hospital.\n")
    
    else:
        if len(nearest_hospital) == 2:
            for near in nearest_hospital:
                count = 0
                point = near

                while visit_dict[point] != []:
                    point = visit_dict[point][0]
                    count += 1

                f.write("The distance to the nearest hospital is " + str(count) + " node(s).\n")

        elif len(nearest_hospital) > 0:
            for near in nearest_hospital:
                count = 0
                point = near

                while visit_dict[point] != []:
                    point = visit_dict[point][0]
                    count += 1
                f.write("The distance to the nearest hospital is " + str(count) + " node(s).\n")
                
            f.write("It is only connected to 1 hospital.\n")

        else:
            f.write("It is not connected to any hospitals.\n")
    
    f.write("\n")
f.close()

### Part(d). Instead of two hospitals, the algorithm will output the top-k nearest hospital from every node, where k is selected by the user.

In [ ]:
%%time
def bfs(visited, graph, node, nearest_hospital, k):
    queue = [] # Initialize a queue
    
    visited.append(node)
    queue.append(node)
    
    if hospital_flag[node]:
        return 1

    while queue:
        s = queue.pop(0) 
        # print (s, end = " ") 

        for neighbour in graph[s]:
            if neighbour not in visited:
                # print(neighbour)
                visit_dict[neighbour].append(s)
                if hospital_flag[neighbour]:
                    nearest_hospital.append(neighbour)
                    if len(nearest_hospital) == k:
                        return 0
                visited.append(neighbour)
                queue.append(neighbour)
    return 0
                
f = open("output(d).txt", "w")

k = int(input("How many hospitals are you searching for? "))

for unit in range(nodes_count):
    # Driver Code
    visited = [] # List to keep track of visited nodes.
    visit_dict = {edge_list: [] for edge_list in range(nodes_count)} # store the parent node of every child node
    nearest_hospital = []
    flag = bfs(visited, edges_dict, unit, nearest_hospital, k)
    
    f.write(str(unit) + "\n")
    
    if flag == 1:
        f.write("It is the hospital.\n")
    
    else:
        if len(nearest_hospital) == k:
            for near in nearest_hospital:
                count = 0
                point = near

                while visit_dict[point] != []:
                    point = visit_dict[point][0]
                    count += 1

                f.write("The distance to the nearest hospital is " + str(count) + " node(s).\n")

        elif len(nearest_hospital) > 0:
            for near in nearest_hospital:
                count = 0
                point = near

                while visit_dict[point] != []:
                    point = visit_dict[point][0]
                    count += 1
                f.write("The distance to the nearest hospital is " + str(count) + " node(s).\n")
                
            f.write("It is only connected to " + str(len(nearest_hospital)) + " hospital(s).\n")

        else:
            f.write("It is not connected to any hospitals.\n")
    
    f.write("\n")
f.close()